In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

for year in range(2018, 2026):
    file_path = f'/content/drive/MyDrive/CS506/MBTA_avg_delay/MBTA_Bus_Arrival_Departure_Times_{year}.zip'
    if os.path.exists(file_path):
        print(f'Unzipping {file_path}...')
        !unzip "{file_path}" -d MBTA_avg_delay
    else:
        print(f'File not found: {file_path}')

In [4]:
!ls MBTA_avg_delay | head -20

Bus Arrival Departure Times Apr-June 2020.csv
Bus Arrival Departure Times Jan-Mar 2020.csv
Bus Arrival Departure Times Jul-Sep 2020.csv
Bus Arrival Departure Times Oct-Dec 2020.csv
__MACOSX
MBTA Bus Arrival Departure Times 2018
MBTA Bus Arrival Departure Times 2019
MBTA_Bus_Arrival_Departure_Times_2021
MBTA_Bus_Arrival_Departure_Times_2022
MBTA_Bus_Arrival_Departure_Times_2023
MBTA_Bus_Arrival_Departure_Times_2024
MBTA_Bus_Arrival_Departure_Times_2025


In [5]:
df18 = pd.read_csv("MBTA_avg_delay/MBTA Bus Arrival Departure Times 2018/MBTA Bus Arrival Departure Aug-Sept 2018.csv")

print(df18.shape)
print(df18.columns)
print(df18.head())

(5117779, 14)
Index(['service_date', 'route_id', 'direction', 'half_trip_id', 'stop_id',
       'time_point_id', 'time_point_order', 'point_type', 'standard_type',
       'scheduled', 'actual', 'earliness', 'scheduled_headway', 'headway'],
      dtype='object')
           service_date route_id direction  half_trip_id  stop_id  \
0  2018-08-01T00:00:00Z       01   Inbound    40121394.0       75   
1  2018-08-01T00:00:00Z       01   Inbound    40121394.0       79   
2  2018-08-01T00:00:00Z       01   Inbound    40121394.0      187   
3  2018-08-01T00:00:00Z       01   Inbound    40121394.0       59   
4  2018-08-01T00:00:00Z       01   Inbound    40121565.0      110   

  time_point_id  time_point_order  point_type standard_type  \
0           mit               4.0    Midpoint      Schedule   
1         hynes               5.0    Midpoint      Schedule   
2         masta               6.0    Midpoint      Schedule   
3         Wasma               7.0    Midpoint      Schedule   
4       

In [6]:
import glob

files = glob.glob("MBTA_avg_delay/**/*.csv", recursive=True)
print("Total files found:", len(files))
print(files[:10]) # Print the first 10 files to verify

Total files found: 67
['MBTA_avg_delay/Bus Arrival Departure Times Oct-Dec 2020.csv', 'MBTA_avg_delay/Bus Arrival Departure Times Jul-Sep 2020.csv', 'MBTA_avg_delay/Bus Arrival Departure Times Jan-Mar 2020.csv', 'MBTA_avg_delay/Bus Arrival Departure Times Apr-June 2020.csv', 'MBTA_avg_delay/MBTA_Bus_Arrival_Departure_Times_2021/MBTA-Bus-Arrival-Departure-Times_2021-10.csv', 'MBTA_avg_delay/MBTA_Bus_Arrival_Departure_Times_2021/MBTA-Bus-Arrival-Departure-Times_2021-08.csv', 'MBTA_avg_delay/MBTA_Bus_Arrival_Departure_Times_2021/MBTA-Bus-Arrival-Departure-Times_2021-11.csv', 'MBTA_avg_delay/MBTA_Bus_Arrival_Departure_Times_2021/MBTA-Bus-Arrival-Departure-Times_2021-12.csv', 'MBTA_avg_delay/MBTA_Bus_Arrival_Departure_Times_2021/MBTA-Bus-Arrival-Departure-Times_2021-01.csv', 'MBTA_avg_delay/MBTA_Bus_Arrival_Departure_Times_2021/MBTA-Bus-Arrival-Departure-Times_2021-06.csv']


In [ ]:
import pandas as pd

columns_per_file = {}

for f in files:
    try:
        # read  the header row
        cols = list(pd.read_csv(f, nrows=0).columns)
        columns_per_file[f.split('/')[-1]] = cols
    except Exception as e:
        print(f"Error reading {f}: {e}")

# summary of cols
for name, cols in columns_per_file.items():
    print(f"\n{name} -> {len(cols)} columns")
    print(cols)


In [7]:
import pandas as pd
import glob
import os

files = glob.glob("MBTA_avg_delay/**/*.csv", recursive=True)
output_dir = "MBTA_avg_delay_standardized"
os.makedirs(output_dir, exist_ok=True)

def standardize_col_names(df):
    new_cols = []
    for col in df.columns:
        c = col.lower().replace(" ", "_")
        mapping = {
            "direction_id": "direction",
            "route": "route_id",
            "service_date": "service_date",
            "timepoint": "time_point_id",
            "stop": "stop_id",
            "halftripid": "half_trip_id",
            "timepointorder": "time_point_order",
            "pointtype": "point_type",
            "standardtype": "standard_type",
            "scheduledheadway": "scheduled_headway",
        }
        new_cols.append(mapping.get(c, c))
    df.columns = new_cols
    if "earliness" in df.columns:
        df = df.drop(columns=["earliness"])
    return df

for f in files:
    try:
        print(f"Processing {f}...")
        df = pd.read_csv(f, dtype=str)  # read as text to avoid type issues
        df = standardize_col_names(df)

        out_path = os.path.join(output_dir, os.path.basename(f))
        df.to_csv(out_path, index=False)
    except Exception as e:
        print(f"Error processing {f}: {e}")

print("All files standardized and saved.")


Processing MBTA_avg_delay/Bus Arrival Departure Times Oct-Dec 2020.csv...
Processing MBTA_avg_delay/Bus Arrival Departure Times Jul-Sep 2020.csv...
Processing MBTA_avg_delay/Bus Arrival Departure Times Jan-Mar 2020.csv...
Processing MBTA_avg_delay/Bus Arrival Departure Times Apr-June 2020.csv...
Processing MBTA_avg_delay/MBTA_Bus_Arrival_Departure_Times_2021/MBTA-Bus-Arrival-Departure-Times_2021-10.csv...
Processing MBTA_avg_delay/MBTA_Bus_Arrival_Departure_Times_2021/MBTA-Bus-Arrival-Departure-Times_2021-08.csv...
Processing MBTA_avg_delay/MBTA_Bus_Arrival_Departure_Times_2021/MBTA-Bus-Arrival-Departure-Times_2021-11.csv...
Processing MBTA_avg_delay/MBTA_Bus_Arrival_Departure_Times_2021/MBTA-Bus-Arrival-Departure-Times_2021-12.csv...
Processing MBTA_avg_delay/MBTA_Bus_Arrival_Departure_Times_2021/MBTA-Bus-Arrival-Departure-Times_2021-01.csv...
Processing MBTA_avg_delay/MBTA_Bus_Arrival_Departure_Times_2021/MBTA-Bus-Arrival-Departure-Times_2021-06.csv...
Processing MBTA_avg_delay/MBTA_

In [9]:
print(df.columns.tolist())

['service_date', 'route_id', 'direction', 'half_trip_id', 'stop_id', 'time_point_id', 'time_point_order', 'point_type', 'standard_type', 'scheduled', 'actual', 'scheduled_headway', 'headway']


In [ ]:
import pandas as pd
import glob
import os
import re

# Your routes of interest
target_routes = {'22', '29', '15', '45', '28', '44', '42', '17', '23', '31', '26', '111', '24', '33', '14'}

files = glob.glob("MBTA_avg_delay_standardized/**/*.csv", recursive=True)

def extract_year_from_filename(path):
    match = re.search(r"(20\d{2})", os.path.basename(path))
    return int(match.group(1)) if match else None

yearly_dfs = []

for f in files:
    try:
        year = extract_year_from_filename(f)
        print(f"Processing {f} (year {year})...")

        # Load only the columns we need
        df = pd.read_csv(f, usecols=["route_id", "scheduled", "actual"], dtype=str)

        # Keep only target routes
        df = df[df["route_id"].isin(target_routes)]

        # Parse times to datetime
        df["scheduled"] = pd.to_datetime(df["scheduled"], errors="coerce")
        df["actual"] = pd.to_datetime(df["actual"], errors="coerce")

        # Compute delay (actual - scheduled) in minutes
        df["delay_min"] = (df["actual"] - df["scheduled"]).dt.total_seconds() / 60

        # Drop missing or unrealistic values
        df = df.dropna(subset=["delay_min"])
        df = df[df["delay_min"].between(-60, 180)]  # optional outlier filter

        # Average delay per route per year
        avg_delay = df.groupby("route_id", as_index=False)["delay_min"].mean()
        avg_delay["year"] = year

        yearly_dfs.append(avg_delay)

    except Exception as e:
        print(f"Error processing {f}: {e}")

# Combine all yearly averages
avg_delay_all = pd.concat(yearly_dfs, ignore_index=True)
avg_delay_all = avg_delay_all.sort_values(["year", "route_id"])

# Save results
avg_delay_all.to_csv("average_delay_target_routes.csv", index=False)

print(avg_delay_all.head(20))


Processing MBTA_avg_delay_standardized/MBTA-Bus-Arrival-Departure-Times_2024-02.csv (year 2024)...
Processing MBTA_avg_delay_standardized/MBTA-Bus-Arrival-Departure-Times_2022-05.csv (year 2022)...
Processing MBTA_avg_delay_standardized/MBTA-Bus-Arrival-Departure-Times_2022-01.csv (year 2022)...
Processing MBTA_avg_delay_standardized/MBTA-Bus-Arrival-Departure-Times_2021-10.csv (year 2021)...
Processing MBTA_avg_delay_standardized/MBTA-Bus-Arrival-Departure-Times_2022-08.csv (year 2022)...
Processing MBTA_avg_delay_standardized/MBTA-Bus-Arrival-Departure-Times_2025-09.csv (year 2025)...
Processing MBTA_avg_delay_standardized/MBTA-Bus-Arrival-Departure-Times_2024-09.csv (year 2024)...
Processing MBTA_avg_delay_standardized/MBTA-Bus-Arrival-Departure-Times_2021-08.csv (year 2021)...
Processing MBTA_avg_delay_standardized/MBTA Bus Arrival Departure Aug-Sept 2018.csv (year 2018)...
Processing MBTA_avg_delay_standardized/Bus Arrival Departure Times Oct-Dec 2020.csv (year 2020)...
Processing